In [9]:
!pip uninstall -y langchain langchain-core langchain-community langchain-google-genai
!pip install langchain==0.1.0 langchain-community==0.0.10 langchain-core==0.1.8 langchain-google-genai==0.0.6 google-search-results
!pip install -q wikipedia

Found existing installation: langchain 0.1.0
Uninstalling langchain-0.1.0:
  Successfully uninstalled langchain-0.1.0
Found existing installation: langchain-core 0.1.8
Uninstalling langchain-core-0.1.8:
  Successfully uninstalled langchain-core-0.1.8
Found existing installation: langchain-community 0.0.10
Uninstalling langchain-community-0.0.10:
  Successfully uninstalled langchain-community-0.0.10
Found existing installation: langchain-google-genai 0.0.6
Uninstalling langchain-google-genai-0.0.6:
  Successfully uninstalled langchain-google-genai-0.0.6
  Using cached langchain-0.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_community-0.0.10-py3-none-any.whl.metadata (7.3 kB)
  Using cached langchain_core-0.1.8-py3-none-any.whl.metadata (4.0 kB)
  Using cached langchain_google_genai-0.0.6-py3-none-any.whl.metadata (2.7 kB)
Using cached langchain-0.1.0-py3-none-any.whl (797 kB)
Using cached langchain_community-0.0.10-py3-none-any.whl (1.5 MB)
Using cached langchain_core-

In [10]:
import wikipedia

In [13]:
wikipedia.summary("Who is virat kohli?",sentences=2)

'Virat Kohli (born 5 November 1988) is an Indian international cricketer and the former all-format captain of the Indian national cricket team. He is a right-handed batter and occasional right-arm medium pace bowler.'

In [14]:
import os
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType
import wikipedia

In [17]:
# --- CONFIGURATION ---
# # 1. Google Gemini Key
os.environ["GOOGLE_API_KEY"] = ""

# # 2. SerpAPI Key (Get free at serpapi.com)
os.environ["SERPAPI_API_KEY"] = ""

# --- CHECK KEYS ---
if not os.environ.get("GOOGLE_API_KEY"):
    print("⚠️ STOP: Please enter your Google API Key!")
else:
    try:
        # --- DEFINE TOOLS ---
      

        @tool
        def search_wiki(query: str):
            """Useful for finding Newton's birth year and death year"""
            print(f"    🔎 [Search Tool] Wiki: '{query}'")
            return wikipedia.summary(query,sentences=2)

        # 2. Calculator
        @tool
        def calculator(expression: str):
            """Useful for calculating math expressions."""
            print(f"    🧮 [Math Tool] Calculating: '{expression}'")
            try:
                return str(eval(expression))
            except:
                return "Error"

        tools = [search_wiki, calculator]

        # --- INITIALIZE AGENT ---
        llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash") # Using generic pro model for compatibility

        # The "Zero Shot ReAct" agent is the standard "Think-Act-Observe" loop
        agent_executor = initialize_agent(
            tools=tools,
            llm=llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True # Helps if the LLM makes a formatting mistake
        )

        # --- RUN TASK ---
        print("\n--- 🤖 REAL AGENT TASK STARTED ---")
        query = "You are fact checker for Science Magzine. A writer has submitted a draft claiming that Isaac Newton was 200 years old.And finally declare the writer's claim is True or False"

        # In older versions, we use .run() instead of .invoke()
        response = agent_executor.run(query)

        print(f"\n--- 🏁 FINAL ANSWER ---\n{response}")

    except Exception as e:
        print(f"\n❌ Error: {e}")


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
E0000 00:00:1766037345.595454 2140773 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



--- 🤖 REAL AGENT TASK STARTED ---


> Entering new AgentExecutor chain...
Action: search_wiki
Action Input: Isaac Newton birth year and death year    🔎 [Search Tool] Wiki: 'Isaac Newton birth year and death year'

Observation: Sir Isaac Newton ( ; 4 January [O.S. 25 December] 1643 – 31 March [O.S. 20 March] 1727) was an English polymath who was a mathematician, physicist, astronomer, alchemist, theologian, author and inventor. He was a key figure in the Scientific Revolution and the Enlightenment that followed.
Thought:Action: calculator
Action Input: 1727 - 1643    🧮 [Math Tool] Calculating: '1727 - 1643'

Observation: 84
Thought:Thought:I have already determined Isaac Newton's age at death by subtracting his birth year from his death year, which is 84 years. The writer claims he was 200 years old. I can now compare these two values to determine if the writer's claim is true or false.
Final Answer:The writer's claim that Isaac Newton was 200 years old is False. Isaac Newton lived for